In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Load the datasets
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

# Data Exploration
print(train_data.info())
print(train_data.describe())

# Handle missing values
train_data['Item_Weight'].fillna(train_data['Item_Weight'].mean(), inplace=True)
test_data['Item_Weight'].fillna(test_data['Item_Weight'].mean(), inplace=True)

train_data['Outlet_Size'].fillna(train_data['Outlet_Size'].mode()[0], inplace=True)
test_data['Outlet_Size'].fillna(test_data['Outlet_Size'].mode()[0], inplace=True)

# Encode categorical variables
label_encoders = {}
categorical_columns = ['Item_Fat_Content', 'Item_Type', 'Outlet_Identifier', 'Outlet_Size', 'Outlet_Location_Type', 'Outlet_Type']

for col in categorical_columns:
    le = LabelEncoder()
    train_data[col] = le.fit_transform(train_data[col])
    test_data[col] = le.transform(test_data[col])
    label_encoders[col] = le

# Feature Engineering
train_data['Outlet_Age'] = 2013 - train_data['Outlet_Establishment_Year']
test_data['Outlet_Age'] = 2013 - test_data['Outlet_Establishment_Year']

# Drop unnecessary columns
train_data.drop(['Item_Identifier', 'Outlet_Establishment_Year'], axis=1, inplace=True)
test_data.drop(['Item_Identifier', 'Outlet_Establishment_Year'], axis=1, inplace=True)

# Split the data into features and target
X = train_data.drop('Item_Outlet_Sales', axis=1)
y = train_data['Item_Outlet_Sales']

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
test_data_scaled = scaler.transform(test_data)

# Model Selection and Training
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Model Validation
y_pred = model.predict(X_val)
rmse = np.sqrt(mean_squared_error(y_val, y_pred))
print(f'RF Validation RMSE: {rmse}')

# Prediction on Test Data
test_predictions = model.predict(test_data_scaled)

# Prepare Submission File
submission = pd.DataFrame({
    'Item_Identifier': pd.read_csv('test.csv')['Item_Identifier'],
    'Outlet_Identifier': pd.read_csv('test.csv')['Outlet_Identifier'],
    'Item_Outlet_Sales': test_predictions
})

submission.to_csv('submission3.csv', index=False)
print("Submission file created successfully!")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB
None
       Item_Weight  Item_Visibility     

### Iteration 4

In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor

# Load the datasets
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

# Feature Engineering
# Extract Item_Category from Item_Identifier
train_data['Item_Category'] = train_data['Item_Identifier'].apply(lambda x: x[:2])
test_data['Item_Category'] = test_data['Item_Identifier'].apply(lambda x: x[:2])

# Bin Item_MRP into price tiers
train_data['Price_Tier'] = pd.cut(train_data['Item_MRP'], bins=[0, 50, 100, 150, 200, 250], labels=[1, 2, 3, 4, 5])
test_data['Price_Tier'] = pd.cut(test_data['Item_MRP'], bins=[0, 50, 100, 150, 200, 250], labels=[1, 2, 3, 4, 5])

# Bin Item_Visibility into categories
train_data['Visibility_Bins'] = pd.cut(train_data['Item_Visibility'], bins=[-1, 0.1, 0.2, 0.3, 1], labels=[1, 2, 3, 4])
test_data['Visibility_Bins'] = pd.cut(test_data['Item_Visibility'], bins=[-1, 0.1, 0.2, 0.3, 1], labels=[1, 2, 3, 4])

# Handle missing values
train_data['Item_Weight'].fillna(train_data['Item_Weight'].mean(), inplace=True)
test_data['Item_Weight'].fillna(test_data['Item_Weight'].mean(), inplace=True)

train_data['Outlet_Age'] = 2013 - train_data['Outlet_Establishment_Year']
test_data['Outlet_Age'] = 2013 - test_data['Outlet_Establishment_Year']

train_data['Outlet_Size'].fillna(train_data['Outlet_Size'].mode()[0], inplace=True)
test_data['Outlet_Size'].fillna(test_data['Outlet_Size'].mode()[0], inplace=True)

# Drop unnecessary columns
train_data.drop(['Item_Identifier', 'Outlet_Establishment_Year'], axis=1, inplace=True)
test_data.drop(['Item_Identifier', 'Outlet_Establishment_Year'], axis=1, inplace=True)

# Log transformation of the target variable
y = np.log1p(train_data['Item_Outlet_Sales'])  # log(1 + x) to handle zero sales
X = train_data.drop('Item_Outlet_Sales', axis=1)

# Define categorical and numerical columns
categorical_cols = ['Item_Fat_Content', 'Item_Type', 'Outlet_Identifier', 'Outlet_Size', 'Outlet_Location_Type', 'Outlet_Type', 'Item_Category', 'Price_Tier', 'Visibility_Bins']
numerical_cols = ['Item_Weight', 'Item_Visibility', 'Item_MRP', 'Outlet_Age']

# Preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ]
)

# Model Pipeline
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor(random_state=42, n_estimators=200, max_depth=6, learning_rate=0.1))
])

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
model.fit(X_train, y_train)

# Validate the model
y_pred = model.predict(X_val)
rmse = np.sqrt(mean_squared_error(y_val, y_pred))
print(f'Validation RMSE: {rmse}')

# Predict on the test data
test_predictions = model.predict(test_data)

# Prepare Submission File
submission = pd.DataFrame({
    'Item_Identifier': pd.read_csv('test.csv')['Item_Identifier'],
    'Outlet_Identifier': pd.read_csv('test.csv')['Outlet_Identifier'],
    'Item_Outlet_Sales': test_predictions
})

submission.to_csv('submission4.csv', index=False)
print("Submission file created successfully!")

Validation RMSE: 0.5337369411719414
Submission file created successfully!


### Iteration 5

In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor
import lightgbm as lgb

# Load the datasets
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

# Feature Engineering
# Extract Item_Category from Item_Identifier
train_data['Item_Category'] = train_data['Item_Identifier'].apply(lambda x: x[:2])
test_data['Item_Category'] = test_data['Item_Identifier'].apply(lambda x: x[:2])

# Bin Item_MRP into price tiers
train_data['Price_Tier'] = pd.cut(train_data['Item_MRP'], bins=[0, 50, 100, 150, 200, 250], labels=[1, 2, 3, 4, 5])
test_data['Price_Tier'] = pd.cut(test_data['Item_MRP'], bins=[0, 50, 100, 150, 200, 250], labels=[1, 2, 3, 4, 5])

# Bin Item_Visibility into categories
train_data['Visibility_Bins'] = pd.cut(train_data['Item_Visibility'], bins=[-1, 0.1, 0.2, 0.3, 1], labels=[1, 2, 3, 4])
test_data['Visibility_Bins'] = pd.cut(test_data['Item_Visibility'], bins=[-1, 0.1, 0.2, 0.3, 1], labels=[1, 2, 3, 4])

# Handle missing values
train_data['Item_Weight'].fillna(train_data['Item_Weight'].mean(), inplace=True)
test_data['Item_Weight'].fillna(test_data['Item_Weight'].mean(), inplace=True)

train_data['Outlet_Age'] = 2013 - train_data['Outlet_Establishment_Year']
test_data['Outlet_Age'] = 2013 - test_data['Outlet_Establishment_Year']

train_data['Outlet_Size'].fillna(train_data['Outlet_Size'].mode()[0], inplace=True)
test_data['Outlet_Size'].fillna(test_data['Outlet_Size'].mode()[0], inplace=True)

# Drop unnecessary columns
train_data.drop(['Item_Identifier', 'Outlet_Establishment_Year'], axis=1, inplace=True)
test_data.drop(['Item_Identifier', 'Outlet_Establishment_Year'], axis=1, inplace=True)

# Log transformation of the target variable
y = np.log1p(train_data['Item_Outlet_Sales'])  # log(1 + x) to handle zero sales
X = train_data.drop('Item_Outlet_Sales', axis=1)

# Define categorical and numerical columns
categorical_cols = ['Item_Fat_Content', 'Item_Type', 'Outlet_Identifier', 'Outlet_Size', 'Outlet_Location_Type', 'Outlet_Type', 'Item_Category', 'Price_Tier', 'Visibility_Bins']
numerical_cols = ['Item_Weight', 'Item_Visibility', 'Item_MRP', 'Outlet_Age']

# Preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ]
)

# Model Pipeline
xgb_model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor(random_state=42, n_estimators=200, max_depth=6, learning_rate=0.1))
])

lgb_model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', lgb.LGBMRegressor(random_state=42, n_estimators=200, max_depth=6, learning_rate=0.1))
])

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Train and evaluate XGBoost model
xgb_model.fit(X_train, y_train)
y_pred_xgb = xgb_model.predict(X_val)
rmse_xgb = np.sqrt(mean_squared_error(y_val, y_pred_xgb))
print(f'XGBoost Validation RMSE: {rmse_xgb}')

# Train and evaluate LightGBM model
lgb_model.fit(X_train, y_train)
y_pred_lgb = lgb_model.predict(X_val)
rmse_lgb = np.sqrt(mean_squared_error(y_val, y_pred_lgb))
print(f'LightGBM Validation RMSE: {rmse_lgb}')

if(rmse_xgb<rmse_lgb):
    model = xgb_model
    print('\nXGBoost')
else:
    model = lgb_model
    print('\nLightGBM')

# Predict on the test data
test_predictions = model.predict(test_data)

# Prepare Submission File
submission = pd.DataFrame({
    'Item_Identifier': pd.read_csv('test.csv')['Item_Identifier'],
    'Outlet_Identifier': pd.read_csv('test.csv')['Outlet_Identifier'],
    'Item_Outlet_Sales': test_predictions
})

submission.to_csv('submission5.csv', index=False)
print("Submission file created successfully!")

XGBoost Validation RMSE: 0.5337369411719414
LightGBM Validation RMSE: 0.5298734232399103

LightGBM
Submission file created successfully!


### Iteration 6

In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Load the datasets
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

# Feature Engineering
# Extract Item_Category from Item_Identifier
train_data['Item_Category'] = train_data['Item_Identifier'].apply(lambda x: x[:2])
test_data['Item_Category'] = test_data['Item_Identifier'].apply(lambda x: x[:2])

# Bin Item_MRP into price tiers
train_data['Price_Tier'] = pd.cut(train_data['Item_MRP'], bins=[0, 50, 100, 150, 200, 250], labels=[1, 2, 3, 4, 5])
test_data['Price_Tier'] = pd.cut(test_data['Item_MRP'], bins=[0, 50, 100, 150, 200, 250], labels=[1, 2, 3, 4, 5])

# Bin Item_Visibility into categories
train_data['Visibility_Bins'] = pd.cut(train_data['Item_Visibility'], bins=[-1, 0.1, 0.2, 0.3, 1], labels=[1, 2, 3, 4])
test_data['Visibility_Bins'] = pd.cut(test_data['Item_Visibility'], bins=[-1, 0.1, 0.2, 0.3, 1], labels=[1, 2, 3, 4])

train_data['Outlet_Age'] = 2013 - train_data['Outlet_Establishment_Year']
test_data['Outlet_Age'] = 2013 - test_data['Outlet_Establishment_Year']

# Handle missing values
train_data['Item_Weight'].fillna(train_data['Item_Weight'].mean(), inplace=True)
test_data['Item_Weight'].fillna(test_data['Item_Weight'].mean(), inplace=True)

train_data['Outlet_Size'].fillna(train_data['Outlet_Size'].mode()[0], inplace=True)
test_data['Outlet_Size'].fillna(test_data['Outlet_Size'].mode()[0], inplace=True)

# Drop unnecessary columns
train_data.drop(['Item_Identifier', 'Outlet_Establishment_Year'], axis=1, inplace=True)
test_data.drop(['Item_Identifier', 'Outlet_Establishment_Year'], axis=1, inplace=True)

# Log transformation of the target variable
y = np.log1p(train_data['Item_Outlet_Sales'])  # log(1 + x) to handle zero sales
X = train_data.drop('Item_Outlet_Sales', axis=1)

# Define categorical and numerical columns
categorical_cols = ['Item_Fat_Content', 'Item_Type', 'Outlet_Identifier', 'Outlet_Size', 'Outlet_Location_Type', 'Outlet_Type', 'Item_Category', 'Price_Tier', 'Visibility_Bins']
numerical_cols = ['Item_Weight', 'Item_Visibility', 'Item_MRP', 'Outlet_Age']

# Preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ]
)

# Model Pipeline
rf_model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(random_state=42))
])

# Hyperparameter tuning using GridSearchCV
param_grid = {
    'regressor__n_estimators': [100, 200],
    'regressor__max_depth': [None, 10, 20, 30],
    'regressor__min_samples_split': [2, 5, 10],
    'regressor__min_samples_leaf': [1, 2, 4],
}

# Use K-Fold Cross-Validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

grid_search = GridSearchCV(rf_model, param_grid, cv=kf, scoring='neg_root_mean_squared_error', n_jobs=-1)
grid_search.fit(X, y)

# Best model from grid search
best_model = grid_search.best_estimator_
print(f'Best Hyperparameters: {grid_search.best_params_}')

# Validate the model using cross-validation
y_pred = best_model.predict(X)
rmse = np.sqrt(mean_squared_error(y, y_pred))
print(f'RF Training RMSE: {rmse}')

# Predict on the test data
test_predictions = best_model.predict(test_data)

# Convert predictions back from log scale
test_predictions = np.expm1(test_predictions)

# Prepare Submission File
submission = pd.DataFrame({
    'Item_Identifier': pd.read_csv('test.csv')['Item_Identifier'],
    'Outlet_Identifier': pd.read_csv('test.csv')['Outlet_Identifier'],
    'Item_Outlet_Sales': test_predictions
})

submission.to_csv('submission6.csv', index=False)
print("Submission file created successfully!")

Best Hyperparameters: {'regressor__max_depth': 10, 'regressor__min_samples_leaf': 1, 'regressor__min_samples_split': 10, 'regressor__n_estimators': 200}
RF Training RMSE: 0.45296043965735666
Submission file created successfully!


### Iteration 7

In [12]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, StackingRegressor
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import lightgbm as lgb

# Load the datasets
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

# Feature Engineering
# Extract Item_Category from Item_Identifier
train_data['Item_Category'] = train_data['Item_Identifier'].apply(lambda x: x[:2])
test_data['Item_Category'] = test_data['Item_Identifier'].apply(lambda x: x[:2])

train_data['Outlet_Age'] = 2013 - train_data['Outlet_Establishment_Year']
test_data['Outlet_Age'] = 2013 - test_data['Outlet_Establishment_Year'] 

# Bin Item_MRP into price tiers
train_data['Price_Tier'] = pd.cut(train_data['Item_MRP'], bins=[0, 50, 100, 150, 200, 250], labels=[1, 2, 3, 4, 5])
test_data['Price_Tier'] = pd.cut(test_data['Item_MRP'], bins=[0, 50, 100, 150, 200, 250], labels=[1, 2, 3, 4, 5])

# Bin Item_Visibility into categories
train_data['Visibility_Bins'] = pd.cut(train_data['Item_Visibility'], bins=[-1, 0.1, 0.2, 0.3, 1], labels=[1, 2, 3, 4])
test_data['Visibility_Bins'] = pd.cut(test_data['Item_Visibility'], bins=[-1, 0.1, 0.2, 0.3, 1], labels=[1, 2, 3, 4])

# Handle missing values
train_data['Item_Weight'].fillna(train_data['Item_Weight'].mean(), inplace=True)
test_data['Item_Weight'].fillna(test_data['Item_Weight'].mean(), inplace=True)

train_data['Outlet_Size'].fillna(train_data['Outlet_Size'].mode()[0], inplace=True)
test_data['Outlet_Size'].fillna(test_data['Outlet_Size'].mode()[0], inplace=True)

# Drop unnecessary columns
train_data.drop(['Item_Identifier', 'Outlet_Establishment_Year'], axis=1, inplace=True)
test_data.drop(['Item_Identifier', 'Outlet_Establishment_Year'], axis=1, inplace=True)

# Log transformation of the target variable
y = np.log1p(train_data['Item_Outlet_Sales'])  # log(1 + x) to handle zero sales
X = train_data.drop('Item_Outlet_Sales', axis=1)

# Define categorical and numerical columns
categorical_cols = ['Item_Fat_Content', 'Item_Type', 'Outlet_Identifier', 'Outlet_Size', 'Outlet_Location_Type', 'Outlet_Type', 'Item_Category', 'Price_Tier', 'Visibility_Bins']
numerical_cols = ['Item_Weight', 'Item_Visibility', 'Item_MRP', 'Outlet_Age']

# Preprocessing pipeline for base models
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ]
)

# Base models for stacking with preprocessing
base_models = [
    ('rf', Pipeline(steps=[('preprocessor', preprocessor), ('regressor', RandomForestRegressor(n_estimators=100, random_state=42))])),
    ('lgb', Pipeline(steps=[('preprocessor', preprocessor), ('regressor', lgb.LGBMRegressor(n_estimators=100, random_state=42))]))
]

# Stacking Regressor
stacked_model = StackingRegressor(estimators=base_models, final_estimator=GradientBoostingRegressor())

# Use K-Fold Cross-Validation for stacking
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Fit the stacked model
stacked_model.fit(X, y)

# Validate the model using cross-validation
y_pred_stacked = stacked_model.predict(X)
rmse_stacked = np.sqrt(mean_squared_error(y, y_pred_stacked))
print(f'Training RMSE for Stacked Model: {rmse_stacked}')

# Predict on the test data
test_predictions = stacked_model.predict(test_data)

# Convert predictions back from log scale
test_predictions = np.expm1(test_predictions)  # Convert back from log(1 + x)

# Prepare Submission File
submission = pd.DataFrame({
    'Item_Identifier': pd.read_csv('test.csv')['Item_Identifier'],
    'Outlet_Identifier': pd.read_csv('test.csv')['Outlet_Identifier'],
    'Item_Outlet_Sales': test_predictions
})

submission.to_csv('submission7.csv', index=False)
print("Submission file created successfully!")

Training RMSE for Stacked Model: 0.46332093562046256
Submission file created successfully!


In [22]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# Load the data
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

# Define the features and the target variable
features = ['Item_Weight', 'Item_Fat_Content', 'Item_Visibility', 'Item_Type', 'Item_MRP', 'Outlet_Identifier', 'Outlet_Establishment_Year', 'Outlet_Size', 'Outlet_Location_Type', 'Outlet_Type']
target = 'Item_Outlet_Sales'

# Split the data into training and testing sets
X_train, y_train = train_df[features], train_df[target]
X_test, y_test = test_df[features], test_df[target]

# Scale the data using MinMaxScaler
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define the neural network model
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu')
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=32)

# Make predictions on the test set
predictions = model.predict(X_test)

# Evaluate the model
rmse = np.sqrt(np.mean((y_test - predictions) ** 2))
print('RMSE:', rmse)

ImportError: 

IMPORTANT: PLEASE READ THIS FOR ADVICE ON HOW TO SOLVE THIS ISSUE!

Importing the numpy C-extensions failed. This error can happen for
many reasons, often due to issues with your setup or how NumPy was
installed.

We have compiled some common reasons and troubleshooting tips at:

    https://numpy.org/devdocs/user/troubleshooting-importerror.html

Please note and check the following:

  * The Python version is: Python3.9 from "C:\Users\INGRSHA\Anaconda3\python.exe"
  * The NumPy version is: "1.21.6"

and make sure that they are the versions you expect.
Please carefully study the documentation linked above for further help.

Original error was: DLL load failed while importing _multiarray_umath: The specified module could not be found.


ImportError: 

IMPORTANT: PLEASE READ THIS FOR ADVICE ON HOW TO SOLVE THIS ISSUE!

Importing the numpy C-extensions failed. This error can happen for
many reasons, often due to issues with your setup or how NumPy was
installed.

We have compiled some common reasons and troubleshooting tips at:

    https://numpy.org/devdocs/user/troubleshooting-importerror.html

Please note and check the following:

  * The Python version is: Python3.9 from "C:\Users\INGRSHA\Anaconda3\python.exe"
  * The NumPy version is: "1.21.6"

and make sure that they are the versions you expect.
Please carefully study the documentation linked above for further help.

Original error was: DLL load failed while importing _multiarray_umath: The specified module could not be found.


ImportError: 

IMPORTANT: PLEASE READ THIS FOR ADVICE ON HOW TO SOLVE THIS ISSUE!

Importing the numpy C-extensions failed. This error can happen for
many reasons, often due to issues with your setup or how NumPy was
installed.

We have compiled some common reasons and troubleshooting tips at:

    https://numpy.org/devdocs/user/troubleshooting-importerror.html

Please note and check the following:

  * The Python version is: Python3.9 from "C:\Users\INGRSHA\Anaconda3\python.exe"
  * The NumPy version is: "1.21.6"

and make sure that they are the versions you expect.
Please carefully study the documentation linked above for further help.

Original error was: DLL load failed while importing _multiarray_umath: The specified module could not be found.


ImportError: _multiarray_umath failed to import

ImportError: numpy._core.umath failed to import

In [20]:
python -m pip install tensorflow



SyntaxError: invalid syntax (1751429654.py, line 1)